# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability for free (ReAct reasoning, memory, multi‑step planning).
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.

## Roadmap
1. Environment setup
2. Write simple tools and connect them to an LLM
3. Standardize tool calling by writing `to_schema`
4. Use LangChain to augment an LLM with your tools
5. Build a Perplexity‑style web‑search agent
6. (Optional) A minimal backend and frontend UI

# 1- Environment setup

## 1.1- Conda environment

Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Once this is done, you can select “web_agent” from the Kernel → Change Kernel menu in Jupyter or VS Code.


> Behind the scenes:
> * Conda reads `environment.yml`, resolves the pinned dependencies, creates an isolated environment named `web_agent`, and activates it.
> * `ollama pull` downloads the model so you can run it locally without API calls.


## 1.2 Ollama setup

In this project, we start with `gemma3-1B` because it is lightweight and runs on most machines. You can try other smaller or larger LLMs such as `mistral:7b`, `phi3:mini`, or `llama3.2:1b` to compare performance. Explore available models here: https://ollama.com/library

```bash
ollama pull gemma3:1b
```

`ollama pull` downloads the model so you can run it locally without API calls.


## 2- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.


We describe each tool’s interface in the model’s prompt, defining what it does and what arguments it expects. When the model decides that a tool is needed, it emits a structured output like: `TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Francisco"}}`. Your code will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3` model how to use it when required in four steps:
1. Implement the tool
2. Create the instructions for the LLM
3. Call the LLM with the prompt
4. Parse the LLM output and call the tool

In [16]:
from openai import OpenAI

client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

In [17]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# Your goal: give the model a way to access weather information.
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Requirements:
#   • The function should be named `get_current_weather`
#   • It should take two arguments:
#         - city: str
#         - unit: str = "celsius"
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#

"""
YOUR CODE HERE (~2-3 lines of code)
"""
import requests


def get_current_weather(city: str, unit: str = "celsius"):
    """Get the current weather in a given city"""
    API_KEY = '2ea40921be0a7ca1ae28b94015d71a2e'

    # OpenWeatherMap uses 'metric' for Celsius, 'imperial' for Fahrenheit
    units = "metric" if unit == "celsius" else "imperial"
    unit_symbol = "°C" if unit == "celsius" else "°F"
    
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units={units}"
    
    try:
        response = requests.get(url)
        data = response.json()
        
        if response.status_code == 200:
            temp = data["main"]["temp"]
            description = data["weather"][0]["description"]
            return f"It is {temp}{unit_symbol} and {description} in {city}."
        else:
            return f"Could not fetch weather for {city}."
    except Exception as e:
        return f"Error fetching weather: {str(e)}"
    

In [18]:
# ---------------------------------------------------------
# Step 2: Create the prompt for the LLM to call tools
# ---------------------------------------------------------
# Goal:
#   Build the system and user prompts that instruct the model when and how
#   to use your tool (`get_current_weather`).
#
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describe the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

# Try experimenting with different system and user prompts
# ---------------------------------------------------------

"""
YOUR CODE HERE
"""
SYSTEM_PROMPT = """
You are a helpful assistant that can answer questions and help with tasks.
You have access to a tool called "get_current_weather" that can give you the current weather in a given city.
To use it, you can call the tool with the city name and unit as arguments, like this:

TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego", "unit": "celsius"}}

Here is the user question:
"""
USER_QUESTION = "What is the weather in Calgary today?"

Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

Start the **Ollama** server in a terminal with `ollama serve`. This launches a local API endpoint that listens for LLM requests. Once the server is running, return to the notebook and in the next cell send a query to the model.


In [19]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Use the Ollama client to create a chat completion. 
#       - You may find some examples here: https://platform.openai.com/docs/guides/text
#       - If you are unsure, search the web for "client.chat.completions.create"
#   2. Print the raw response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------

"""
YOUR CODE HERE (~5-10 lines of code)
"""
response = client.chat.completions.create(
    model="mistral:7b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_QUESTION}
    ]
)

print(response.choices[0].message.content)

 To find out the current weather in Calgary, I can use the tool "get_current_weather". Here's how I will call it:

TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Calgary", "unit": "celsius"}}

Once the tool returns the data, I can tell you the weather in Calgary today.


In [20]:
# ---------------------------------------------------------
# Step 4: Parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json

"""
YOUR CODE HERE (~5-10 lines of code)
"""
import re
import json

def extract_tool_call_json(text):
    # Find the start of the JSON
    match = re.search(r'TOOL_CALL:\s*(\{)', text)
    if not match:
        return None
    
    start_pos = match.start(1)
    brace_count = 0
    in_string = False
    escape_next = False
    
    for i, char in enumerate(text[start_pos:], start=start_pos):
        if escape_next:
            escape_next = False
            continue
        
        if char == '\\':
            escape_next = True
            continue
            
        if char == '"':
            in_string = not in_string
        elif not in_string:
            if char == '{':
                brace_count += 1
            elif char == '}':
                brace_count -= 1
                if brace_count == 0:
                    return text[start_pos:i+1]
    
    return None


TOOL_MAP = {
    "get_current_weather": get_current_weather
}

response_string = response.choices[0].message.content

# Usage:
tool_call_json = extract_tool_call_json(response_string)
print(tool_call_json)

if tool_call_json:
    tool_call = json.loads(tool_call_json)
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]
    print(f"Calling tool `{tool_name}` with args {tool_args}")
    result = TOOL_MAP[tool_name](**tool_args)
    print(result)


{"name": "get_current_weather", "args": {"city": "Calgary", "unit": "celsius"}}
Calling tool `get_current_weather` with args {'city': 'Calgary', 'unit': 'celsius'}
It is 7.74°C and broken clouds in Calgary.


# 3- Standadize tool calling

So far, we handled tool calling manually by writing one regex and one hard-coded function. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the `TOOL_SPEC` prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function’s signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [21]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Use `inspect.signature` to get function parameters.
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing:
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect


def to_schema(fn):
    """
    YOUR CODE HERE (~8-15 lines of code)
    """
    sig = inspect.signature(fn)
    json_dict = {"tool_name": fn.__name__, "args": []}
    for param in sig.parameters.values():
        json_dict["args"].append({
            "name": param.name,
            "type": str(param.annotation),
            "default": param.default
        })
    return json_dict


import random
import string

def print_random_string(length=8):
    rand_str = ''.join(random.choices(string.ascii_uppercase, k=length))
    print(rand_str)


functions = [
    get_current_weather,
    print_random_string,
]

full_tool_schema = []
for fn in functions:
    full_tool_schema.append(to_schema(fn))
pprint(full_tool_schema)

[{'args': [{'default': <class 'inspect._empty'>,
            'name': 'city',
            'type': "<class 'str'>"},
           {'default': 'celsius', 'name': 'unit', 'type': "<class 'str'>"}],
  'tool_name': 'get_current_weather'},
 {'args': [{'default': 8,
            'name': 'length',
            'type': "<class 'inspect._empty'>"}],
  'tool_name': 'print_random_string'}]


In [22]:
# ---------------------------------------------------------
# Provide the tool schema to the model
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the raw model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

import json
tool_spec = json.dumps(full_tool_schema, default=str)
print(tool_spec)

"""
YOUR CODE HERE (~5-12 lines of code)
"""
SYSTEM_PROMPT_2 = """
You are a helpful assistant that can answer questions and help with tasks.
You have access to a list of tools that you can use to answer the user's question.
To use a tool, you can call the tool by providing the tool name and the arguments, like this:

TOOL_CALL: {{"name": "get_current_weather", "args": {{"city": "San Diego", "unit": "celsius"}}}}

Here is the list of tools available to you:
{tool_spec}

Here is the user question:
""".format(tool_spec=tool_spec)

USER_QUESTION = "What is the weather in Calgary today?"


[{"tool_name": "get_current_weather", "args": [{"name": "city", "type": "<class 'str'>", "default": "<class 'inspect._empty'>"}, {"name": "unit", "type": "<class 'str'>", "default": "celsius"}]}, {"tool_name": "print_random_string", "args": [{"name": "length", "type": "<class 'inspect._empty'>", "default": 8}]}]


In [23]:
response = client.chat.completions.create(
    model="mistral:7b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_2},
        {"role": "user", "content": USER_QUESTION}
    ]
)

print(response.choices[0].message.content)

 To answer the user's question, I will call the `get_current_weather` tool with the argument `city` set to 'Calgary' and default unit as 'Celsius'. The response will contain information about the current weather in Calgary.

TOOL_CALL: {"name": "get_current_weather", "args": {"city": "Calgary", "unit": "celsius"}}


## 4- LangChain for Tool Calling
So far, you built a simple tool-calling pipeline manually. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward.

In this section, you will use the **ReAct** Agent (Reasoning + Acting). It alternates between reasoning steps and tool use, producing clearer and more reliable results. We will explore reasoning-focused models in more depth next week.

The following links might be helpful:
- https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html
- https://python.langchain.com/docs/integrations/tools/
- https://python.langchain.com/docs/integrations/chat/ollama/
- https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.llms.LLM.html

In [24]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Goal:
#   Convert your weather function into a LangChain-compatible tool.
#
# Steps:
#   1. Import `tool` from `langchain.tools`.
#   2. Keep your existing `get_current_weather` helper as before.
#   3. Create a new function (e.g., get_weather) that calls it.
#   4. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.
#   • Start with keeping the logic simple and offline-friendly.

from langchain.tools import tool

"""
YOUR CODE HERE (~5 lines of code)
"""
@tool
def get_weather(city: str, unit: str = "celsius"):
    """Gets the weather of the given city in the given units"""
    return get_current_weather(city, unit)

In [25]:
# ---------------------------------------------------------
# Step 2: Initialize the LangChain Agent
# ---------------------------------------------------------
# Goal:
#   Connect your tool to a local LLM using LangChain’s ReAct-style agent.
#
# Steps:
#   1. Import the required classes:
#        - ChatOllama (for local model access)
#        - initialize_agent, Tool, AgentType
#   2. Create an LLM instance (e.g., model="gemma3:1b", temperature=0).
#   3. Add your tool(s) to a list
#   4. Initialize the agent using initialize_agent
#   5. Test the agent with a natural question (e.g., "Do I need an umbrella in Seattle today?").
#
# Expected:
#   The model should reason through the question, call your tool,
#   and produce a final answer in plain language.
# ---------------------------------------------------------

from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType

"""
YOUR CODE HERE (~5 lines of code)
"""
llm = ChatOllama(model="mistral:7b", temperature=0)
tools = [get_weather]

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent_executor.invoke("I'm in San Diego - should I bring an umbrella today?")




> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "get_weather",
  "action_input": {
    "city": "San Diego",
    "unit": "celsius"
  }
}
```

Observation: It is 20.22°C and broken clouds in San Diego.
Thought: Based on the weather report, it seems like there's a chance of some cloud cover but no heavy rain forecasted for today in San Diego. However, it's always good to be prepared. I would recommend carrying an umbrella just in case, but you might not need to use it extensively.

Action:
```
{
  "action": "Final Answer",
  "action_input": "Given the weather forecast, it's recommended to carry an umbrella as there's a chance of cloud cover, but heavy rain is unlikely."
}
```

> Finished chain.


{'input': "I'm in San Diego - should I bring an umbrella today?",
 'output': "Given the weather forecast, it's recommended to carry an umbrella as there's a chance of cloud cover, but heavy rain is unlikely."}

### What just happened?

The console log displays the **Thought → Action → Observation → …** loop until the agent produces its final answer. Because `verbose=True`, LangChain prints each intermediate reasoning step.

If you want to add more tools, simply append them to the tools list. LangChain will handle argument validation, schema generation, and tool-calling logic automatically.


## 5- Perplexity‑Style Web Search
Agents become much more powerful when they can look up real information on the web instead of relying only on their internal knowledge.

In this section, you will combine everything you have learned to build a simple Ask-the-Web Agent. You will integrate a web search tool (DuckDuckGo) and make it available to the agent using the same tool-calling approach as before.

This will let the model retrieve fresh results, reason over them, and generate an informed answer—similar to how Perplexity works.

You may find some examples from the following links:
- https://pypi.org/project/duckduckgo-search/

In [28]:
# ---------------------------------------------------------
# Step 1: Add a web search tool
# ---------------------------------------------------------
# Goal:
#   Create a tool that lets the agent search the web and return results.
#
# Steps:
#   1. Use DuckDuckGo for quick, open web searches.
#   2. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DDGS to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   3. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS
from langchain.tools import tool

"""
YOUR CODE HERE (~5-10 lines of code)
"""
@tool
def search_web(query: str):
    """Searches the web for the given query"""
    results = DDGS().text(query, max_results=5)
    return results

results = search_web("python programming")
print("Results:")
print(results)
print("Done!")

/var/folders/v4/rz21fv5s5fn_4nv27jvlbh3m0000gn/T/ipykernel_35891/1810273572.py:28: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = search_web("python programming")


Results:
[{'title': 'Welcome to Python.org', 'href': 'https://www.python.org/', 'body': 'Python is a versatile and easy-to-learn programming language that lets you work quickly and integrate systems more effectively. Learn Python basics, download the latest version, access documentation, find jobs, events, success stories and more on the official website.'}, {'title': 'Python Tutorial - W3SchoolsUsage example', 'href': 'https://www.w3schools.com/python/', 'body': 'W3Schools offers a comprehensive and interactive Python tutorial with examples, exercises, quizzes, and references. Learn how to create web applications, handle files and databases, and get certified by completing the PYTHON course. See more on w3schools'}, {'title': 'Learn Python Programming', 'href': 'https://www.programiz.com/python-programming', 'body': 'A comprehensive guide to learn Python, one of the top programming languages in the world, widely used in AI, data science, and web development. Find free tutorials, inter

In [33]:

# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Goal:
#   Connect your `web_search` tool to a language model
#   so the agent can search and reason over real data.
#
# Steps:
#   1. Import `initialize_agent` and `AgentType`.
#   2. Create an LLM (e.g., ChatOllama).
#   3. Add your `web_search` tool to the tools list.
#   4. Initialize the agent using: initialize_agent
#   5. Keep `verbose=True` to observe reasoning steps.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI

"""
YOUR CODE HERE (~5 lines of code)
"""
llm = ChatOllama(model="mistral:7b", temperature=0)
tools_list = [get_weather, search_web]
agent_executor = initialize_agent(
    tools_list,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



Let’s see the agent's output in action with a real example.


In [34]:

# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent
# ---------------------------------------------------------
# Goal:
#   Verify that the agent can search the web and return
#   a summarized answer based on real results.
#
# Steps:
#   1. Ask a natural question that requires live information,
#      for example: "What are the current events in San Francisco this week?"
#   2. Call agent.
#
# Expected:
#   The agent should call `web_search`, retrieve results,
#   and generate a short summary response.
# ---------------------------------------------------------

"""
YOUR CODE HERE (~2-5 lines of code)
"""

agent_executor.invoke("What is the weather in New York today and how will it affect the stock market?")




> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "get_weather",
  "action_input": {
    "city": "New York",
    "unit": "celsius"
  }
}
```

Observation: It is 17.13°C and few clouds in New York.
Thought: Now I need to find out if the weather in New York today will affect the stock market. To do this, I'll search the web for information on how weather affects the stock market, specifically focusing on New York.

Action:
```
{
  "action": "search_web",
  "action_input": {
    "query": "how does weather in New York affect the stock market"
  }
}
```


Observation: [{'title': 'Data Study on Weather vs. theStockMarket', 'href': 'https://nycdatascience.com/blog/student-works/data-study-on-weather-vs-the-stock-market/', 'body': 'Doesweatheraffectthe financial markets? Is there a correlation between the two? Dostock prices and equities trading transaction volumes follow a pattern in relation to different weather conditions?'}, {'title': 'DoesWeatherAffecttheStockMarket?', 'h

{'input': 'What is the weather in New York today and how will it affect the stock market?',
 'output': 'I have provided you with links to articles that discuss the potential impact of weather on the stock market, specifically focusing on New York. You can further investigate these sources if you wish.'}


## 6- A minimal UI
This project includes a simple **React** front end that sends the user’s question to a FastAPI back end and streams the agent’s response in real time. To run the UI:

1- Open a terminal and start the Ollama server: `ollama serve`.

2- In a second terminal, navigate to the frontend folder and install dependencies:`npm install`.

3- In the same terminal, start the FastAPI back‑end: `uvicorn app:app --reload --port 8000`

4- Open a third terminal, stay in the frontend folder, and start the React dev server: `npm run dev`

5- Visit `http://localhost:5173/` in your browser.



## 🎉 Congratulations!

* You have built a **web‑enabled agent**: tool calling → JSON schema → LangChain ReAct → web search → simple UI.
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.


👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.